<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=deb602fb70614bb68decb941bcd021796e21666a7b68d137661e968021b941d3
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 10:42:00
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -28.34 K (-0.2%)
Current PnL: -23.60 L (-15.46%)
CY Booked + Current PnL: -9.71 L (-6.36%)
-------------------
Total profit:  1.31 L
Total loss:  -24.91 L
-------------------
Total Booked + Current PnL: 17.27 L (13.75%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.77%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.01 L (62.6%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.25,-14.01,16.39,0.09,14204.0,-14114.0,86663.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.86,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.52,-9.80,10.89,0.02,17146.0,-17110.0,157449.0,139.96,68.0,M-SC,6.64,234.0,-1.00,1.11,36.95,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,0.72,7.34,11.98,20.19,22415.0,12787.0,187107.0,-15.78,52.0,X-MC,7.61,68.0,0.57,1.32,23.56,X40,ATH,INSURANCE
43,ITC,452.00,0.15,-1.25,11.84,10.44,23399.0,-2509.0,197629.0,-41.15,44.0,X-LC,0.82,5.0,-0.11,1.39,4.24,X40,NTT,FMCG
51,MASFIN,398.61,0.23,-4.99,28.46,22.05,26493.0,-4890.0,93090.0,-18.25,50.0,H-SC,7.35,164.0,-0.18,0.65,35.31,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,3.98,-49.33,233.64,69.05,157527.0,-65643.0,67423.0,-58.98,38.0,H-SC,23.55,140.0,-0.42,0.47,3.98,SR,ATH,IT
61,REPCOHOME,880.00,2.76,-10.59,112.30,89.82,287669.0,-30344.0,256161.0,-55.29,50.0,H-SC,10.43,160.0,-0.11,1.80,35.53,XY24,NTT,FINANCE
56,QUESS,424.00,2.40,-27.43,96.82,42.84,45677.0,-17829.0,47177.0,-52.62,41.0,X-SC,37.35,83.0,-0.39,0.33,2.65,XY24,NTT,MISC
64,SFL,1287.00,2.09,-39.95,110.86,26.63,174571.0,-104748.0,157470.0,19.19,33.0,M-SC,4.50,239.0,-0.60,1.11,2.09,XY24,NTT,MISC
18,COFFEEDAY,80.00,1.93,-38.30,119.24,35.27,83541.0,-43488.0,70061.0,-51.71,36.0,L-SC,17.78,270.0,-0.52,0.49,71.48,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-5.15,-16.67,84.20,53.50,211723.0,-50299.0,251452.0,-59.67,41.0,X-MC,4.07,58.0,-0.24,1.77,11.69,XY24,NTT,FMCG
34,HONAUT,58357.33,-3.45,-14.01,66.66,43.32,93364.0,-22812.0,140060.0,-26.09,43.0,X-SC,2.01,90.0,-0.24,0.99,7.54,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-2.53,-21.61,109.41,64.15,204072.0,-51431.0,186520.0,-54.13,53.0,X-SC,1.65,82.0,-0.25,1.31,4.63,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-1.84,-63.70,175.68,0.07,87478.0,-87383.0,49794.0,1724.49,54.0,L-SC,1.51,268.0,-1.00,0.35,23.84,OX40N,NTT,JEWELLERY
29,HAPPSTMNDS,1480.71,-1.66,-37.77,197.03,84.85,157151.0,-48405.0,79760.0,-25.98,43.0,H-SC,14.22,147.0,-0.31,0.56,2.37,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.16,-0.68,108.13,106.72,132458.0,-837.0,122499.0,-50.82,33.0,H-SC,3.70,139.0,-0.01,0.86,22.54,AR,ATH,MISC
85,WIPRO,420.00,-0.83,0.57,71.28,72.27,121476.0,973.0,170421.0,-13.79,53.0,M-LC,4.56,99.0,0.01,1.20,6.78,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.07,-5.21,119.34,107.91,167018.0,-7693.0,139951.0,-22.98,33.0,M-SC,10.48,216.0,-0.05,0.98,0.07,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.52,-9.80,10.89,0.02,17146.0,-17110.0,157449.0,139.96,68.0,M-SC,6.64,234.0,-1.00,1.11,36.95,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.25,-14.01,16.39,0.09,14204.0,-14114.0,86663.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.86,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.75,-23.16,47.67,13.47,98780.0,-62451.0,207216.0,-68.51,25.0,H-SC,1.38,158.0,-0.63,1.46,5.36,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.33,-23.46,30.63,-0.01,29964.0,-29979.0,97826.0,19921.74,55.0,M-SC,1.67,233.0,-1.00,0.69,10.77,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.83,0.57,71.28,72.27,121476.0,973.0,170421.0,-13.79,53.0,M-LC,4.56,99.0,0.01,1.2,6.78,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.83,0.57,71.28,72.27,121476.0,973.0,170421.0,-13.79,53.0,M-LC,4.56,99.0,0.01,1.20,6.78,XR,NTT,IT
38,INDIAMART,4810.62,0.16,-0.68,108.13,106.72,132458.0,-837.0,122499.0,-50.82,33.0,H-SC,3.70,139.0,-0.01,0.86,22.54,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.59,-1.70,37.98,35.63,78241.0,-3569.0,206005.0,-15.41,37.0,H-MC,4.39,119.0,-0.05,1.45,14.69,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.07,-5.21,119.34,107.91,167018.0,-7693.0,139951.0,-22.98,33.0,M-SC,10.48,216.0,-0.05,0.98,0.07,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.14,-5.31,92.13,81.92,100143.0,-6100.0,108698.0,-44.55,32.0,M-SC,4.52,236.0,-0.06,0.76,28.68,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.56,-15.57,68.40,42.18,159629.0,-43034.0,233376.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.0,-0.30,-35.32,109.04,35.20,90747.0,-45446.0,83224.0,9.17,24.0,X-SC,12.88,92.0,-0.50,0.59,0.23,X40,NTT,FOOTWEAR
46,JSWINFRA,342.0,-0.06,-9.42,27.21,15.23,49379.0,-18873.0,181474.0,-22.54,24.0,X-MC,6.65,66.0,-0.38,1.28,19.79,X40N,NTT,REALTY
60,RELAXO,1176.0,-0.12,-47.88,192.25,52.31,145439.0,-69509.0,75651.0,-44.54,27.0,X-SC,3.95,91.0,-0.48,0.53,1.26,X40N,NTT,FOOTWEAR
27,GLAXO,3466.2,-1.19,-1.75,41.63,39.15,81508.0,-3492.0,195792.0,-21.11,29.0,X-MC,9.47,60.0,-0.04,1.38,26.74,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.56,-15.57,68.40,42.18,159629.0,-43034.0,233376.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,0.15,-1.25,11.84,10.44,23399.0,-2509.0,197629.0,-41.15,44.0,X-LC,0.82,5.0,-0.11,1.39,4.24,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.59,1.28,29.31,30.97,52467.0,2270.0,179008.0,-22.52,49.0,X-MC,1.57,75.0,0.04,1.26,14.01,X40,ATH,INSURANCE
3,ACC,3906.0,-2.53,-21.61,109.41,64.15,204072.0,-51431.0,186520.0,-54.13,53.0,X-SC,1.65,82.0,-0.25,1.31,4.63,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.29,-14.74,47.22,25.51,74919.0,-27435.0,158660.0,-2.66,54.0,H-LC,1.87,49.0,-0.37,1.12,16.92,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.56,-15.57,68.40,42.18,159629.0,-43034.0,233376.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.02,-13.51,71.65,48.45,124251.0,-27092.0,173414.0,-19.66,31.0,X-MC,6.81,64.0,-0.22,1.22,0.20,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.30,-35.32,109.04,35.20,90747.0,-45446.0,83224.0,9.17,24.0,X-SC,12.88,92.0,-0.50,0.59,0.23,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-0.35,-5.40,33.26,26.07,51520.0,-8840.0,154900.0,-30.31,34.0,X-MC,9.03,55.0,-0.17,1.09,0.41,X40,ATH,APPARELS
55,PGHH,17907.65,-0.07,-4.19,39.61,33.76,76212.0,-8415.0,192405.0,-32.38,34.0,X-MC,3.35,57.0,-0.11,1.35,0.79,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,2.40,-27.43,96.82,42.84,45677.0,-17829.0,47177.0,-52.62,41.0,X-SC,37.35,83.0,-0.39,0.33,2.65,XY24,NTT,MISC
60,RELAXO,1176.00,-0.12,-47.88,192.25,52.31,145439.0,-69509.0,75651.0,-44.54,27.0,X-SC,3.95,91.0,-0.48,0.53,1.26,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.30,-35.32,109.04,35.20,90747.0,-45446.0,83224.0,9.17,24.0,X-SC,12.88,92.0,-0.50,0.59,0.23,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-1.40,2.95,20.52,24.08,26819.0,3748.0,130698.0,-5.33,45.0,X-SC,4.96,89.0,0.14,0.92,23.15,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.27,-6.04,29.40,21.58,38711.0,-8469.0,131670.0,-52.64,31.0,X-MC,6.84,56.0,-0.22,0.93,16.57,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.27,-14.88,40.13,19.28,118172.0,-51482.0,294474.0,-25.89,60.0,X-LC,7.52,1.0,-0.44,2.07,8.86,X40,ATH,IT
41,INFY,2275.00,-0.77,4.57,48.10,54.87,157378.0,14301.0,327189.0,-18.52,57.0,X-LC,3.00,2.0,0.09,2.30,13.23,X40,BTT,IT
76,TMPV,600.00,-0.56,-15.57,68.40,42.18,159629.0,-43034.0,233376.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.45,-9.48,49.89,35.67,142630.0,-29954.0,285888.0,-20.30,42.0,X-LC,5.95,4.0,-0.21,2.01,3.06,X40N,ATH,FMCG
43,ITC,452.00,0.15,-1.25,11.84,10.44,23399.0,-2509.0,197629.0,-41.15,44.0,X-LC,0.82,5.0,-0.11,1.39,4.24,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,1.25,-12.15,106.79,81.67,88011.0,-11395.0,82415.0,7411.11,54.0,L-SC,18.81,271.0,-0.13,0.58,59.60,XR,NTT,CERAMICS
68,SONACOMS,806.63,-1.18,-13.53,61.37,39.54,53683.0,-13685.0,87474.0,-32.36,62.0,M-SC,6.69,220.0,-0.25,0.62,23.35,AR,ATH,AUTO
51,MASFIN,398.61,0.23,-4.99,28.46,22.05,26493.0,-4890.0,93090.0,-18.25,50.0,H-SC,7.35,164.0,-0.18,0.65,35.31,XR,ATH,FINANCE
70,SURYODAY,216.00,0.28,-19.60,53.03,23.03,76349.0,-35098.0,143973.0,57.89,48.0,H-SC,10.61,167.0,-0.46,1.01,42.85,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.52,-9.80,10.89,0.02,17146.0,-17110.0,157449.0,139.96,68.0,M-SC,6.64,234.0,-1.00,1.11,36.95,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.52,-9.80,10.89,0.02,17146.0,-17110.0,157449.0,139.96,68.0,M-SC,6.64,234.0,-1.00,1.11,36.95,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,0.77,1.41,19.26,20.95,49175.0,3555.0,255323.0,0.35,78.0,X-LC,13.43,31.0,0.07,1.80,37.55,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.40,-38.52,116.16,32.89,55134.0,-29742.0,47464.0,-686.14,58.0,L-MC,5.55,259.0,-0.54,0.33,30.71,XR,NTT,BANKS
12,BERGEPAINT,680.00,-0.21,2.99,17.63,21.14,41277.0,6791.0,234130.0,-4.29,63.0,X-MC,3.91,74.0,0.16,1.65,31.97,XY24,NTT,PAINTS
79,UNITDSPR,1644.00,0.13,5.27,14.72,20.77,36281.0,12332.0,246476.0,-3.83,60.0,X-MC,3.57,62.0,0.34,1.73,12.55,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.61
2,50,75.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.61
MC    29.24
LC    25.13
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.93
X40      21.21
X40N     11.90
XR        9.61
XY25      9.07
AR        8.98
OX40N     7.66
X200      1.86
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.91
X-MC    22.81
X-LC    19.98
M-SC    12.04
X-SC     8.24
H-MC     4.72
H-LC     2.98
L-SC     1.42
M-MC     1.38
M-LC     1.20
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.73,-5.77,39.98
IT,13.73,-15.73,75.47
FINANCE,9.48,-14.96,63.54
MISC,7.42,-23.07,73.48
PAINTS,6.02,-8.10,24.16
ELECTRICAL,5.92,-11.75,52.68
INSURANCE,4.45,-0.02,35.27
PHARMA,3.87,-1.63,34.16
AUTO,3.34,-21.01,72.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3063360.0,21
AR,1297359.0,10
XR,1291823.0,13
X40,1024049.0,14
X40N,756504.0,9
OX40N,702590.0,10
XY25,352649.0,6
SR,278865.0,2
MH,73949.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3574627.0,25
M-SC,1345999.0,15
X-MC,1242602.0,16
X-LC,920057.0,11
X-SC,748466.0,8
H-MC,407511.0,3
L-SC,259030.0,3
H-LC,134769.0,2
M-LC,121476.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218395.0      6
           AR         881392.0      5
           XR         793850.0      7
M-SC       XY24       784401.0      6
X-MC       X40        448375.0      7
X-LC       X40        399022.0      5
X-MC       XY24       360780.0      3
H-SC       OX40N      328176.0      4
X-SC       X40N       295246.0      3
M-SC       OX40N      286936.0      5
H-SC       SR         278865.0      2
X-SC       XY24       276568.0      3
X-MC       X40N       248622.0      4
X-LC       XY24       231902.0      2
H-MC       AR         216197.0      2
X-LC       X40N       212636.0      2
H-MC       XY24       191314.0      1
X-MC       XY25       184825.0      2
X-SC       X40        176652.0      2
L-SC       XR         171552.0      2
M-SC       XR         149811.0      2
           AR         124851.0      2
M-LC       XR         121476.0      1
L-SC       OX40N       87478.0      1
X-LC       XY25        76497.0      2
H-LC       AR          74919.0      1
H-SC       MH          73949.0      1
H-LC       X200        59850.0      1
L-MC       XR          55134.0      1
M-MC       XY25        49781.0      1
L-LC       XY25        41546.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
